# Виконання

## Основне завдання

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
df = pd.read_csv('bbc-news-data.csv', sep='\t')
df

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...
...,...,...,...,...
2220,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...
2221,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...
2222,tech,399.txt,Be careful how you code,A new European directive could put software w...
2223,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...


*Зчитування файлу*

### Видалимо колонки 'filename', 'title', 'category'.

In [2]:
df.drop(['filename', 'title', 'category'], axis=1, inplace=True)
df

,content
0,Quarterly profits at US media giant TimeWarne...
1,The dollar has hit its highest level against ...
2,The owners of embattled Russian oil giant Yuk...
3,British Airways has blamed high fuel prices f...
4,Shares in UK drinks and food firm Allied Dome...
...,...
2220,BT is introducing two initiatives to help bea...
2221,Computer users across the world continue to i...
2222,A new European directive could put software w...
2223,The man making sure US computer networks are ...


*Видалення колонок*

### Видалимо порожні документи, якщо вони є.

In [3]:
df = df[~(df.content.str.strip() == '')]
df

,content
0,Quarterly profits at US media giant TimeWarne...
1,The dollar has hit its highest level against ...
2,The owners of embattled Russian oil giant Yuk...
3,British Airways has blamed high fuel prices f...
4,Shares in UK drinks and food firm Allied Dome...
...,...
2220,BT is introducing two initiatives to help bea...
2221,Computer users across the world continue to i...
2222,A new European directive could put software w...
2223,The man making sure US computer networks are ...


*Видалення порожніх документів*

### Визначимо стоп-слова англійської мови.

In [4]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

*Стоп-слова*

### Визначимо функцію, що виконує попередню обробку документу. Застосуємо декоратор np.vectorize для того, щоб функція могла працювати з корпусами.

In [5]:
@np.vectorize
def preproc_doc(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I | re.A)
    doc = doc.lower()
    doc = doc.strip()
    tokens = wpt.tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens)
    return doc

p_corpus = preproc_doc(df.content)
p_corpus

array(['quarterly profits us media giant timewarner jumped bn three months december yearearlier firm one biggest investors google benefited sales highspeed internet connections higher advert sales timewarner said fourth quarter sales rose bn bn profits buoyed oneoff gains offset profit dip warner bros less users aol time warner said friday owns searchengine google internet business aol mixed fortunes lost subscribers fourth quarter profits lower preceding three quarters however company said aols underlying profit exceptional items rose back stronger internet advertising revenues hopes increase subscribers offering online service free timewarner internet customers try sign aols existing customers highspeed broadband timewarner also restate results following probe us securities exchange commission sec close concluding time warners fourth quarter profits slightly better analysts expectations film division saw profits slump helped boxoffice flops alexander catwoman sharp contrast yearearli

*Обробка документів*

### Розіб'ємо кожний документ на окремі слова, об'єднаємо усі слова в одну сукупність.

In [6]:
words = []
for doc in p_corpus:
    words.extend(doc.split(' '))
df_words = pd.DataFrame(set(words))
df_words.columns = ['words']
df_words.head(10)

,words
0,exeter
1,sunner
2,sisterhood
3,barrels
4,afterwards
5,shopfronts
6,along
7,contradictory
8,hogged
9,seagram


*Розбиття на слова*

### Визначимо частину мови для кожного слова. Використаємо функцію nltk.pos_tag.

In [7]:
df_words['ps'] = [tag for _, tag in nltk.pos_tag(df_words.words)]
df_words

,words,ps
0,exeter,NN
1,sunner,NN
2,sisterhood,NN
3,barrels,NNS
4,afterwards,VBP
...,...,...
31333,forcibly,RB
31334,challengers,VBZ
31335,suffers,NNS
31336,amaya,JJ


*Визначення частини мови*

### Узагальнимо частини мови до звичайних: noun, adective, verb тощо.

In [8]:
from nltk.tag import map_tag
df_words['sps'] = [map_tag('en-ptb', 'universal', tag) for tag in df_words.ps]
df_words

,words,ps,sps
0,exeter,NN,NOUN
1,sunner,NN,NOUN
2,sisterhood,NN,NOUN
3,barrels,NNS,NOUN
4,afterwards,VBP,VERB
...,...,...,...
31333,forcibly,RB,ADV
31334,challengers,VBZ,VERB
31335,suffers,NNS,NOUN
31336,amaya,JJ,ADJ


*Узагальнення чатин мов*

### Перетворимо узагальнені частини мови на абревіатури для лематизації.

In [9]:
abbr = {'NOUN': 'n', 'VERB': 'v', 'ADJ': 'a', 'ADV': 'r'}
def to_abbr(el):
    res = abbr.get(el)
    if res is not None:
        return res
    return ''

df_words['abbr'] = [to_abbr(x) for x in df_words.sps]
df_words

,words,ps,sps,abbr
0,exeter,NN,NOUN,n
1,sunner,NN,NOUN,n
2,sisterhood,NN,NOUN,n
3,barrels,NNS,NOUN,n
4,afterwards,VBP,VERB,v
...,...,...,...,...
31333,forcibly,RB,ADV,r
31334,challengers,VBZ,VERB,v
31335,suffers,NNS,NOUN,n
31336,amaya,JJ,ADJ,a


*Приведення загальних частин мов до абревіатур*

### Проведемо лематизацію кожного слова за допомогою методу lemmatize об'єкта класу nltk.stem.WordNetLemmatizer.

In [10]:
from nltk.stem import WordNetLemmatizer
wlem = WordNetLemmatizer()
def my_lem(word, abbr):
    if abbr == '':
        return wlem.lemmatize(word)
    return wlem.lemmatize(word, pos=abbr)
df_words['lemms'] = [my_lem(row[1]['words'], row[1]['abbr']) 
               for row in df_words.loc[:, ['words', 'abbr']].iterrows()]
df_words

,words,ps,sps,abbr,lemms
0,exeter,NN,NOUN,n,exeter
1,sunner,NN,NOUN,n,sunner
2,sisterhood,NN,NOUN,n,sisterhood
3,barrels,NNS,NOUN,n,barrel
4,afterwards,VBP,VERB,v,afterwards
...,...,...,...,...,...
31333,forcibly,RB,ADV,r,forcibly
31334,challengers,VBZ,VERB,v,challengers
31335,suffers,NNS,NOUN,n,suffers
31336,amaya,JJ,ADJ,a,amaya


*Лематизація слів*

### Представимо корпус як модуль "Сумка слів". Використаємо для цього клас CountVectorizer зі sklearn.feature_extraction.text.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(p_corpus)
cv_matrix = pd.DataFrame(cv_matrix.toarray(), 
                         columns=cv.get_feature_names_out())
cv_matrix.to_csv('bag_of_words.csv')

*Сумка слів*

### Представимо корпус як модель TD-IDF. Перетворимо матрицю з частотою термінів на матрицю tfidf.

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tt = TfidfTransformer(norm='l2', use_idf=True)
tt_matrix = tt.fit_transform(cv_matrix)
tt_matrix = tt_matrix.toarray()
vocab = cv.get_feature_names_out()
tv_matrix = pd.DataFrame(np.round(tt_matrix, 2), columns=vocab)
tv_matrix

,00,000,05,10,100,11,12,125,13,14,...,zooropa,zornotza,zorro,zubair,zuluaga,zurich,zurichs,zutons,zvonareva,zvyagintsev
0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2221,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2222,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2223,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


*Матриця TF-IDF*

### Підрахуємо частоту кожного слова.

In [13]:
df_counts = pd.DataFrame(nltk.FreqDist(words).items(), columns=['words', 'counts'])
df_counts.set_index('words', inplace=True)
df_counts

,counts
words,
quarterly,21
profits,152
us,1807
media,309
giant,152
...,...
191,1
trifling,1
24hours,1


*Частоти слів*

### Відсортуємо датафрейм за спаданням частоти.

In [14]:
df_counts.sort_values(by='counts', ascending=False, inplace=True)
df_counts

,counts
words,
said,7252
mr,3004
would,2574
also,2156
people,1968
...,...
braced,1
symbologist,1
brotherhood,1


*Сортований датафрейм*

### Зобразимо перші десять найбільш уживаних слів.

In [15]:
df_counts.head(10)

,counts
words,
said,7252
mr,3004
would,2574
also,2156
people,1968
new,1901
us,1807
one,1732
year,1624


*Найуживаніші слова*

### Виведемо метрику для перших десяти елементів

In [16]:
tv_matrix[df_counts.index[:10].to_list()]

,said,mr,would,also,people,new,us,one,year,could
0,0.05,0.00,0.00,0.03,0.00,0.00,0.06,0.02,0.00,0.00
1,0.03,0.02,0.00,0.00,0.00,0.04,0.16,0.00,0.02,0.05
2,0.04,0.00,0.02,0.00,0.00,0.00,0.05,0.00,0.00,0.00
3,0.06,0.02,0.02,0.01,0.00,0.00,0.00,0.00,0.07,0.00
4,0.02,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.02,0.02
...,...,...,...,...,...,...,...,...,...,...
2220,0.03,0.00,0.02,0.00,0.04,0.04,0.00,0.00,0.00,0.00
2221,0.04,0.00,0.00,0.02,0.09,0.00,0.00,0.02,0.00,0.00
2222,0.00,0.00,0.01,0.00,0.01,0.07,0.04,0.04,0.02,0.02
2223,0.01,0.10,0.00,0.04,0.02,0.00,0.10,0.02,0.03,0.02


*Метрики для найбільш уживаних слів*

## Додаткове завдання 1

## Зчитаємо текст.

In [17]:
df_ukr = pd.read_csv('ukr_text.csv')
df_ukr

,Id,Title,Body
0,http://k.img.com.ua/rss/ua/4013798,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...
1,http://k.img.com.ua/rss/ua/4001679,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...
2,http://k.img.com.ua/rss/ua/4001390,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...
3,http://k.img.com.ua/rss/ua/4001239,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...
4,http://k.img.com.ua/rss/ua/4001227,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...
...,...,...,...
1117,http://k.img.com.ua/rss/ua/3194862,Корреспондент: Діамантові руки. Історія успіху...,Київський офіс Класичного ювелірного дому Лобо...
1118,http://k.img.com.ua/rss/ua/3194633,Корреспондент: Роздача слонів. Янукович щедро ...,20 років тому орден За заслуги – тоді він нази...
1119,http://k.img.com.ua/rss/ua/3194587,Корреспондент: Рівняння з трьома відомими. Укр...,10 жовтня політичні важковаговики з табору опо...
1120,http://k.img.com.ua/rss/ua/3194570,Корреспондент: Точка зору. Мета обкрадає кошти...,"Добре там, де нас немає. В Ізраїлі ми є, але т..."


*Зчитування файлу*

### Видалимо колонки 'id'.

In [18]:
df_ukr.drop(['Id'], axis=1, inplace=True)
df_ukr

,Title,Body
0,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...
1,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...
2,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...
3,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...
4,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...
...,...,...
1117,Корреспондент: Діамантові руки. Історія успіху...,Київський офіс Класичного ювелірного дому Лобо...
1118,Корреспондент: Роздача слонів. Янукович щедро ...,20 років тому орден За заслуги – тоді він нази...
1119,Корреспондент: Рівняння з трьома відомими. Укр...,10 жовтня політичні важковаговики з табору опо...
1120,Корреспондент: Точка зору. Мета обкрадає кошти...,"Добре там, де нас немає. В Ізраїлі ми є, але т..."


*Видалення колонок*

### Видалимо порожні документи, якщо вони є.

In [19]:
df_ukr = df_ukr[~(df.content.str.strip() == '')]
df_ukr

/tmp/ipykernel_12985/2724321335.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_ukr = df_ukr[~(df.content.str.strip() == '')]


,Title,Body
0,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...
1,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...
2,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...
3,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...
4,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...
...,...,...
1117,Корреспондент: Діамантові руки. Історія успіху...,Київський офіс Класичного ювелірного дому Лобо...
1118,Корреспондент: Роздача слонів. Янукович щедро ...,20 років тому орден За заслуги – тоді він нази...
1119,Корреспондент: Рівняння з трьома відомими. Укр...,10 жовтня політичні важковаговики з табору опо...
1120,Корреспондент: Точка зору. Мета обкрадає кошти...,"Добре там, де нас немає. В Ізраїлі ми є, але т..."


*Видалення порожніх документів*

### Визначимо стоп-слова української мови. Завантажимо їх.

In [20]:
import requests
url = 'https://raw.githubusercontent.com/olegdubetcky/Ukrainian-Stopwords/main/ukrainian'
r = requests.get(url)
with open(nltk.data.path[0]+'/corpora/stopwords/ukrainian', 'wb') as f:
    f.write(r.content)

*Завантаження стоп-слів*

### Визначимо стоп-слова.

In [21]:
import string
import pymorphy2
from nltk.corpus import stopwords
stop_words = stopwords.words("ukrainian")
morph = pymorphy2.MorphAnalyzer(lang='uk')
stop_words = pd.Series(list(set(stop_words+list(string.punctuation))))
stop_words

0         чи
1         по
2         ні
3       чого
4        від
       ...  
182    знову
183      мій
184     буде
185        "
186      щоб
Length: 187, dtype: object

*Визначення стоп-слів*

### Завантажимо корпус.

In [26]:
p_corpus_ukr = preproc_doc(df_ukr.Body)
p_corpus_ukr

array(['еруть участь близько 1000 чоловік , серед яких представники понад 50 промислово - торговельних палат німеччини , провідних німецьких компаній , які здійснюють зовнішньоторговельну діяльність , експерти . " сьогодні київ , як ніколи , підготовлений і відкритий до співпраці . перспективний для інвестування ", - сказав кличко . він зазначив , що 60 % всіх іноземних інвестицій в українську економіку " приходять " саме в столицю , адже в місті введені прозорі механізми управління , а " гарантом " для інвесторів мер виступає особисто . кличко запевнив , що київ є надійним партнером і перспективним ринком для бізнесу та інвестицій . " хочу запевнити , ми - надійні партнери . київ - перспективний ринок для бізнесу та інвестицій . і його мер - захисник інвестицій і добре говорить по - німецьки . ми приречені на ефективну співпрацю . і налаштовані на неї !" - звернувся до представників німецького бізнесу кличко . він зазначив , що для україни одним з пріоритетних напрямків є розвиток від

*Корпус*

### Завантажимо тональний словник української мови.

In [27]:
import csv
url = 'https://raw.githubusercontent.com/lang-uk/tone-dict-uk/master/tone-dict-uk.tsv'
r = requests.get(url)
with open(nltk.data.path[0]+'/tone-dict-uk.tsv', 'wb') as f:
    f.write(r.content)
    
d = {}
with open(nltk.data.path[0]+'/tone-dict-uk.tsv', 'r') as csv_file:
    for row in csv.reader(csv_file, delimiter='\t'):
        d[row[0]] = float(row[1])
        
from nltk.sentiment.vader import SentimentIntensityAnalyzer
SIA = SentimentIntensityAnalyzer()
SIA.lexicon.update(d)

*Завантаження тонального словника*

### Порахуємо оцінку настрою для тексту.

In [28]:
df_ukr['score'] = df_ukr.apply(
    lambda row: SIA.polarity_scores(row.Body)["compound"], axis = 1)
df_ukr

,Title,Body,score
0,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...,0.8268
1,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...,-0.2500
2,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...,-0.6124
3,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...,-0.2500
4,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...,-0.2500
...,...,...,...
1117,Корреспондент: Діамантові руки. Історія успіху...,Київський офіс Класичного ювелірного дому Лобо...,0.9584
1118,Корреспондент: Роздача слонів. Янукович щедро ...,20 років тому орден За заслуги – тоді він нази...,0.9949
1119,Корреспондент: Рівняння з трьома відомими. Укр...,10 жовтня політичні важковаговики з табору опо...,0.7980
1120,Корреспондент: Точка зору. Мета обкрадає кошти...,"Добре там, де нас немає. В Ізраїлі ми є, але т...",-0.8385


*Оцінка настрою*